Please note that the xpaths will change according to your google form and so might the structure of the anchor tags being scraped according to the news webnite.

In [ ]:
#import the required libraries and put in the static information
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from openai import OpenAI
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('-incognito')
driver = webdriver.Chrome()
driver.get('google form link goes here')
name="your name"
client = OpenAI(api_key='your api key goes here')

#changeable information
date="date"
newspaper="newspaper name"
urlWeb = 'news website link goes here'

# Send a GET request to the URL
responsetotal = requests.get(urlWeb)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(responsetotal.content, 'html.parser')

# Find all the 'a' tags within the 'div' tag with class 'a-section'
anchors = soup.find_all('a',href=lambda href: href and href.startswith("http://timesofindia.indiatimes.com/"))

# Extract the href attribute for each 'a' tag

# Print each URL
count=1
for anchor in anchors:
    if(count<=100):
        url=anchor.get('href')
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the news article
        title=soup.find('h1',class_='HNMDR')
        content_t = soup.find('div', class_='_s30J clearfix')
        links = content_t.find_all('a')
        # Iterate through the links
        for link in links:
            # Check if the link text contains the specific text for ad removal
            if "Explore Your Financial Landscape with Personalized Credit Insights" in link.get_text():
                # Remove the link from the content
                link.extract()

        # Extract the remaining content
        content = content_t.get_text().strip()
        # Extract the title and content of the article
        print(count)
        headline=title.text.strip()
        content=content_t.text.strip()
        time.sleep(5)

         #summary calling functions
        response = client.chat.completions.create(
          model="gpt-3.5-turbo-0125",
          messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "please summarize the below articles in about 100 words"},
            {"role": "assistant", "content": "Sure please provide the article you want to summarize"},
              {"role": "user", "content": content},
          ]
        )
        summary=response.choices[0].message.content
        response2 = client.chat.completions.create(
          model="gpt-3.5-turbo-0125",
            response_format={ "type": "json_object" },
          messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "please give me a suitable category from the list below for each article i will post afterwards"},
            {"role": "assistant", "content": "Sure please provide the category list."},
              {"role": "user", "content": "Here is the list of categories I want to use : Politics, International News, National News, Local News, Politics, Business and Finance, Science and Technology, Health and Wellness, Entertainment, Sports, Lifestyle and Features, Opinion and Editorial, Environment, Education, Crime and Justice, Human Interest, Obituaries, Weather, Religion and Spirituality, Technology and Gadgets, Automotive"},
              {"role": "assistant", "content": "Thank you for providing the list of categories. Please share the article and I will help you determine the suitable category for it."},
               {"role": "user", "content":content},
          ]
        )
        categ=response2.choices[0].message.content
        category_temp = json.loads(categ)
        category=category_temp['category']

        #filling form begins

        name_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input'))
        )
        name_field.send_keys(name)


        newspaper_field = WebDriverWait(driver,5 ).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input'))
        )
        newspaper_field.send_keys(newspaper)


        date_field = WebDriverWait(driver,5 ).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div/div[2]/div[1]/div/div[1]/input'))
        )
        date_field.send_keys(date)


        headline_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[4]/div/div/div[2]/div/div[1]/div/div[1]/input'))
        )
        headline_field.send_keys(headline)


        content_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[1]/div[2]/textarea'))
        )
        content_field.send_keys(content)


        summary_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[6]/div/div/div[2]/div/div[1]/div[2]/textarea'))
        )
        summary_field.send_keys(summary)


        category_xpath='//*[@id="i39"]/div[3]/div'
        if category == 'Politics':
            category_xpath = '//*[@id="i30"]/div[3]/div'
        elif category == 'International News':
            category_xpath = '//*[@id="i33"]/div[3]/div'
        elif category == 'National News':
            category_xpath = '//*[@id="i36"]/div[3]/div'
        elif category == 'Local News':
            category_xpath = '//*[@id="i39"]/div[3]/div'
        elif category == 'Business and Finance':
            category_xpath = '//*[@id="i45"]/div[3]/div'
        elif category == 'Science and Technology':
            category_xpath = '//*[@id="i48"]/div[3]/div'
        elif category == 'Health and Wellness':
            category_xpath = '//*[@id="i51"]/div[3]/div'
        elif category == 'Entertainment':
            category_xpath = '//*[@id="i54"]/div[3]/div'
        elif category == 'Sports':
            category_xpath = '//*[@id="i57"]/div[3]/div'
        elif category == 'Lifestyle and Features':
            category_xpath = '//*[@id="i60"]/div[3]/div'
        elif category == 'Opinion and Editorial':
            category_xpath = '//*[@id="i63"]/div[3]/div'
        elif category == 'Environment':
            category_xpath = '//*[@id="i66"]/div[3]/div'
        elif category == 'Education':
            category_xpath = '//*[@id="i69"]/div[3]/div'
        elif category == 'Crime and Justice':
            category_xpath = '//*[@id="i72"]/div[3]/div'
        elif category == 'Human Interest':
            category_xpath = '//*[@id="i75"]/div[3]/div'
        elif category == 'Obituaries':
            category_xpath = '//*[@id="i78"]/div[3]/div'
        elif category == 'Weather':
            category_xpath = '//*[@id="i81"]/div[3]/div'
        elif category == 'Religion and Spirituality':
            category_xpath = '//*[@id="i84"]/div[3]/div'
        elif category == 'Technology and Gadgets':
            category_xpath = '//*[@id="i87"]/div[3]/div'
        elif category == 'Automotive':
            category_xpath = '//*[@id="i90"]/div[3]/div'

        category_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH,category_xpath))
        )
        category_field.click()



        submit_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div[1]/div'))
        )
        submit_field.click()


        confirmation_field = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'vHW8K'))
        )
        if(confirmation_field.text=="Your response has been recorded."):
            print("done successfully")
            print(headline)
            print(summary)
            print(category)
            submit_another_response_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div/div[4]/a'))
            )
            submit_another_response_link.click()
            count+=1
        else : print("unsuccessfull")
    else : break